In [38]:
from dolfin import *

\begin{align}
-\Delta u = \lambda u, \quad \Omega\\
u = 0, \quad \partial \Omega
\end{align}

In [39]:
mesh = UnitSquareMesh(32, 32)
V = FunctionSpace(mesh, "Lagrange", 1)
def boundary(x):
    return x[0] < DOLFIN_EPS or x[0] > 1.0 - DOLFIN_EPS or x[1] < DOLFIN_EPS or x[1] > 1.0 - DOLFIN_EPS

v0 = Constant(0.0)
bc = DirichletBC(V, v0, boundary)

In [40]:
u = TrialFunction(V)
v = TestFunction(V)

In [41]:
dummy = inner(Constant(1), v)*dx
a = inner(grad(u), grad(v))*dx
asm = SystemAssembler(a, dummy, bc)
A = PETScMatrix(); asm.assemble(A)

In [42]:
b = inner(u, v)*dx
asm = SystemAssembler(b, dummy)
B = PETScMatrix(); asm.assemble(B)
bc.zero(B)

In [43]:
num_eigenfunctions = 45

In [44]:
%%time
solver = SLEPcEigenSolver(A, B)
solver.parameters["solver"] = "krylov-schur"
solver.parameters["spectrum"] = "target magnitude"
solver.parameters["problem_type"] = "gen_hermitian"
solver.parameters["spectral_transform"] = "shift-and-invert"
solver.parameters["spectral_shift"] = 10.
solver.solve(num_eigenfunctions)

CPU times: user 62.5 ms, sys: 0 ns, total: 62.5 ms
Wall time: 53.9 ms


In [45]:
solver.get_eigenpair(0)

(19.786792290191517,
 0.0,
 <dolfin.cpp.la.PETScVector at 0x7fdcdd243970>)

In [46]:
eigenmodes = File("Eigen/eigenmodes.pvd")
eigenfunction = Function(V, name="eigenfunction")
print(solver.get_number_converged())
for i in range(solver.get_number_converged()):
    (r, _, rv, _) = solver.get_eigenpair(i)
    eigenfunction.vector().zero()
    eigenfunction.vector().axpy(1, rv)
    eigenmodes << eigenfunction
    File(f'Eigen/eigenfunction_{i}.xml')  << eigenfunction

49
